# Casos de uso API football-data.org

In [ ]:
# Importar las librerías necesarias
import requests
from datetime import datetime

# Reemplaza TU_API_KEY con tu clave de la API de football-data.org. Puedes obtener una clave gratuita registrándote en su sitio web
API_KEY = "TU_API_KEY"

headers = {"X-Auth-Token": API_KEY}

A continuación se muestra un ejemplo de un llamado a uno de los endpoints de la categoría Jugadores de la API football-data.org. Se pueden usar otras urls que se encuentran en el pdf "Actividad_Colaborativa_M5.docx"

In [7]:
# Codigo base para llamar los distintos endpoints de la API football-data.org

# Ejemplo enpoint para obtener los partidos de un jugador (Partidos de Buffon en la temporada 2019)
url= "https://api.football-data.org/v4/persons/2019/matches?status=FINISHED"

response = requests.get(url, headers=headers)
response.json()
    

{'filters': {'limit': 15,
  'offset': 0,
  'competitions': 'SA,ESC,FL1',
  'permission': 'TIER_ONE',
  'status': ['FINISHED']},
 'resultSet': {'count': 0},
 'aggregations': 'As this a CPU intensive operation, this is only available for paid subscriptions.',
 'person': {'id': 2019,
  'name': 'Gianluigi Buffon',
  'firstName': 'Gianluigi',
  'lastName': 'Buffon',
  'dateOfBirth': '1978-01-28',
  'nationality': 'Italy',
  'section': 'Goalkeeper',
  'position': None,
  'shirtNumber': None,
  'lastUpdated': '2023-07-30T17:08:52Z'},
 'matches': []}

### Caso de uso 1

🔎 Análisis de Rendimiento de un Equipo en una Competencia

Objetivo: Obtener el rendimiento detallado de un equipo específico en una liga.

Endpoints 
1.	GET /v4/competitions/{id}/standings – Para saber la posición del equipo.
2.	GET /v4/teams/{id}/matches – Para listar todos los partidos jugados.
3.	GET /v4/matches/{id} – Para extraer estadísticas detalladas de cada partido.

Flujo:
1.	Seleccionar la competencia (ej. Premier League).
2.	Buscar el equipo en los standings.
3.	Traer todos los partidos del equipo.
4.	Analizar los partidos individualmente (goles, tarjetas, posesión). Para este paso se necesita ser usuario premium de pago


In [8]:
# Parámetros que puedes modificar
COMPETITION_ID = "PL" 
SEASON = "2024"  

In [9]:
# Paso 1: Obtener la tabla de posiciones (standings) de la competencia
teams_url = f"https://api.football-data.org/v4/competitions/{COMPETITION_ID}/teams?season={SEASON}"
teams_resp = requests.get(teams_url, headers=headers).json()

# Mostrar todos los equipos para verificar el nombre y el ID
for table in teams_resp.get("teams", []):
    print(f"{table['id']} - {table['name']}")

57 - Arsenal FC
58 - Aston Villa FC
61 - Chelsea FC
62 - Everton FC
63 - Fulham FC
64 - Liverpool FC
65 - Manchester City FC
66 - Manchester United FC
67 - Newcastle United FC
73 - Tottenham Hotspur FC
76 - Wolverhampton Wanderers FC
338 - Leicester City FC
340 - Southampton FC
349 - Ipswich Town FC
351 - Nottingham Forest FC
354 - Crystal Palace FC
397 - Brighton & Hove Albion FC
402 - Brentford FC
563 - West Ham United FC
1044 - AFC Bournemouth


In [4]:
# Elegir un equipo específico para analizar
# Puedes cambiar el nombre del equipo aquí
TEAM_NAME = "Manchester City FC"

# Obtener standings
standings_url = f"https://api.football-data.org/v4/competitions/{COMPETITION_ID}/standings?season={SEASON}"
standings_resp = requests.get(standings_url, headers=headers).json()

# Buscar el equipo
for table in standings_resp.get("standings", []):
    if table.get("type") == "TOTAL":
        for team in table.get("table", []):
            if team["team"]["name"] == TEAM_NAME:
                team_id = team["team"]["id"]
                position = team["position"]
                points = team["points"]
                print(f"✅ El {team_id} {TEAM_NAME} está en la posición {position} con {points} puntos.")
                break

if not team_id:
    print(f"❌ Equipo '{TEAM_NAME}' no encontrado.")


✅ El 65 Manchester City FC está en la posición 3 con 71 puntos.


In [9]:
# Paso 2: Obtener todos los partidos jugados por el equipo seleccionado
matches_url = f"https://api.football-data.org/v4/teams/{team_id}/matches?season={SEASON}&status=FINISHED&competitions={COMPETITION_ID}"
matches_resp = requests.get(matches_url, headers=headers).json()
matches = matches_resp.get("matches", [])

print(f"📊 Total de partidos jugados: {len(matches)}")

📊 Total de partidos jugados: 38


In [ ]:
# Paso 3: Analizar los primeros 5 partidos (puedes cambiar el número)
# No se encuentran el equipo en la lista de partidos ya que se necesita ser usuario premium de pago.
for match in matches[:5]:
    match_id = match["id"]
    match_url = f"https://api.football-data.org/v4/matches/{match_id}"
    match_detail = requests.get(match_url, headers=headers).json()

    m = match_detail.get("match", {})
    home = m.get("homeTeam", {}).get("name", "N/A")
    away = m.get("awayTeam", {}).get("name", "N/A")
    score = m.get("score", {}).get("fullTime", {})
    home_goals = score.get("home", 0)
    away_goals = score.get("away", 0)

    print(f"\n⚽ {home} {home_goals} - {away_goals} {away}")

    # Mostrar estadísticas si están disponibles (no todas las APIs gratuitas las ofrecen)
    stats = m.get("statistics", [])
    if stats:
        for stat in stats:
            if stat["team"]["id"] == team_id:
                for s in stat["statistics"]:
                    print(f"   - {s['type']}: {s['value']}")
    else:
        print("   🔍 Estadísticas detalladas no disponibles para este partido.")
    print("-" * 40)


⚽ N/A 0 - 0 N/A
   🔍 Estadísticas detalladas no disponibles para este partido.
----------------------------------------

⚽ N/A 0 - 0 N/A
   🔍 Estadísticas detalladas no disponibles para este partido.
----------------------------------------

⚽ N/A 0 - 0 N/A
   🔍 Estadísticas detalladas no disponibles para este partido.
----------------------------------------

⚽ N/A 0 - 0 N/A
   🔍 Estadísticas detalladas no disponibles para este partido.
----------------------------------------

⚽ N/A 0 - 0 N/A
   🔍 Estadísticas detalladas no disponibles para este partido.
----------------------------------------


### Caso de uso 2

📊 Perfil de Jugador

Objetivo: Construir un perfil técnico completo de un jugador.

Endpoints:
1.	GET /v4/teams/{id} – Para obtener el equipo al que pertenece.
2.	GET /v4/teams/{id} (respuesta contiene squad) – Para encontrar al jugador por ID.
3.	GET /v4/players/{id} – Para obtener los detalles personales y rendimiento (los datos de rendimiento son de la versión de pago).

Flujo:
1.	Identificar el equipo al que pertenece el jugador.
2.	Desde el equipo obtener su plantilla.
3.	Seleccionar el jugador y consultas sus datos.


In [12]:
# 🏟️ Cambia esto por el ID del equipo que quieres analizar
TEAM_ID = 65  # Ejemplo: Manchester City

In [ ]:
# Paso 1: Obtener información del equipo
team_url = f"https://api.football-data.org/v4/teams/{TEAM_ID}"
team_resp = requests.get(team_url, headers=headers).json()

print(f"🏟️ Equipo: {team_resp.get('name')}")
print(f"🌍 País: {team_resp.get('area', {}).get('name')}")
print(f"🛡️ Siglas: {team_resp.get('tla')}")

🏟️ Equipo: Manchester City FC
🌍 País: England
🛡️ Siglas: MCI


In [ ]:
# Paso 2: Mostrar la plantilla del equipo para verificar el nombre y el ID
for table in team_resp.get("squad", []):
    print(f"{table['id']} - {table['name']}")

3222 - Ederson
3953 - Marcus Bettinelli
6958 - Stefan Ortega Moreno
153874 - James Trafford
180453 - Sverre Nypan
206743 - Nico O'Reilly
38101 - Erling Haaland
10183 - Rúben Dias
8642 - Rayan Aït Nouri
8240 - Nathan Aké
7888 - Phil Foden
7696 - Tijjani Reijnders
4147 - Kalvin Phillips
3895 - Jack Grealish
3313 - John Stones
3254 - Bernardo Silva
3199 - Rodri
3182 - Ilkay Gündogan
143 - Manuel Akanji
65 - Mateo Kovačić
212868 - Abdukodir Khusanov
206908 - Claudio Echeverri
187202 - Rico Lewis
179716 - Oscar Bobb
172956 - James McAtee
172791 - Josh Wilson-Esbrand
171986 - Nico Gonzalez
146352 - Sávio
133242 - Rayan Cherki
123574 - Issa Kaboré
122266 - Joško Gvardiol
99775 - Jeremy Doku
91512 - Matheus Nunes
66896 - Omar Marmoush


In [20]:
# Paso 3: Seleccionar el Id de un jugador específico para analizar usando la lista de jugadores del paso 2
player_id = '38101'

if player_id:
    player_url = f"https://api.football-data.org/v4/persons/{player_id}"
    player_resp = requests.get(player_url, headers=headers).json()
    
    print(f"😎 Nombre: {player_resp.get('name')}")
    print(f"🎂 Fecha de nacimiento: {player_resp.get('dateOfBirth')}")
    print(f"👕 Número: {player_resp.get('shirtNumber')}")
    print(f"⚽ Posición: {player_resp.get('section')}")
    print(f"🌍 Nacionalidad: {player_resp.get('nationality')}")
    print(f"📃 Contrato: Desde {player_resp.get('currentTeam', {}).get('contract', {}).get('start')} hasta {player_resp.get('currentTeam', {}).get('contract', {}).get('until')}")
    


😎 Nombre: Erling Haaland
🎂 Fecha de nacimiento: 2000-07-21
👕 Número: 9
⚽ Posición: Centre-Forward
🌍 Nacionalidad: Norway
📃 Contrato: Desde 2022-07 hasta 2034-06



###  Caso de Uso 3: 

📅 Calendario y Resultados de una Competencia

Objetivo: Mostrar todos los partidos de una competición con resultados actualizados.

1.	GET /v4/competitions/{id}/matches – Para obtener el calendario completo.
2.	GET /v4/matches/{id} – Para profundizar en cada partido (estadísticas y eventos con usuario de pago).
3.	GET /v4/teams/{id} – Para mostrar escudos, nombres, y datos de los equipos en los partidos.

Flujo:
1.	El usuario selecciona una liga (ej. La Premier).
2.	Se listan los partidos, fechas y resultados.
3.	El usuario puede elegir un partido para ver estadísticas y jugadores destacados.

In [3]:
# 🏆 Competencia que deseas consultar
COMPETITION_ID = "PL"  # LaLiga

In [ ]:
# Paso 1: Obtener partidos con su respectivo Id
matches_url = f"https://api.football-data.org/v4/competitions/{COMPETITION_ID}/matches?season=2024"
matches_resp = requests.get(matches_url, headers=headers).json()

matches = matches_resp.get("matches", [])

# mostramos los primeros 10
for i, match in enumerate(matches[:10], start=1):  

    home_team = match["homeTeam"]["name"]
    away_team = match["awayTeam"]["name"]
    print(f"{i}. {home_team} vs {away_team} (Match_Id: {match['id']})")

1. Manchester United FC vs Fulham FC (Match_Id: 497410)
2. Ipswich Town FC vs Liverpool FC (Match_Id: 497411)
3. Arsenal FC vs Wolverhampton Wanderers FC (Match_Id: 497412)
4. Everton FC vs Brighton & Hove Albion FC (Match_Id: 497413)
5. Newcastle United FC vs Southampton FC (Match_Id: 497414)
6. Nottingham Forest FC vs AFC Bournemouth (Match_Id: 497415)
7. West Ham United FC vs Aston Villa FC (Match_Id: 497416)
8. Brentford FC vs Crystal Palace FC (Match_Id: 497417)
9. Chelsea FC vs Manchester City FC (Match_Id: 497418)
10. Leicester City FC vs Tottenham Hotspur FC (Match_Id: 497419)


In [14]:
# Paso 2: Seleccionar un partido y mostrar detalles (las estadísticas no estan disponibles en la versión gratuita de la API)

# Cambiar este ID por el que se quiera ver
match_id = 497412

match_detail_url = f"https://api.football-data.org/v4/matches/{match_id}"
match_detail_resp = requests.get(match_detail_url, headers=headers).json()

# Extraer detalles del partido, fecha y hora
print(f"🏟️ {match_detail_resp.get('homeTeam', {}).get('name')} vs {match_detail_resp.get('awayTeam', {}).get('name')}")
date_obj = datetime.fromisoformat(match_detail_resp.get('utcDate').replace("Z", "+00:00"))
print(f"📆 Fecha: {date_obj.date()}")
print(f"⏰ Hora(UTC): {date_obj.time()}")
# Extraer el resultado del partido
home_team = match_detail_resp["homeTeam"]["tla"]
away_team = match_detail_resp["awayTeam"]["tla"]
score = match_detail_resp.get("score", {}).get("fullTime", {})
home_score = score.get("home", "-")
away_score = score.get("away", "-")
print(f"⚽ Resultado: {home_team} {home_score} : {away_team} {away_score}")
# Extraer información del referee
referee = match_detail_resp.get("referees", [{}])[0].get("name")
nationality = match_detail_resp.get("referees", [{}])[0].get("nationality")
print(f"🚩 Árbitro: {referee} - 🌍 Nacionalidad: {nationality}")

🏟️ Arsenal FC vs Wolverhampton Wanderers FC
📆 Fecha: 2024-08-17
⏰ Hora(UTC): 14:00:00
⚽ Resultado: ARS 2 : WOL 0
🚩 Árbitro: Jarred Gillett - 🌍 Nacionalidad: Australia
